In [1]:
# Load in necessary libraries
import os
import glob
import pandas as pd
import numpy as np
from pyhdf.SD import SD, SDC
import geopandas as gpd
from shapely.geometry import Point
from rasterio.io import MemoryFile
from rasterio.mask import mask
from rasterio.transform import from_origin
from pyproj import CRS
import re


In [2]:
# EDIT INPUTS BASED ON SITE OF INTEREST

# Single site to process
SITE_NAME = "MexicoAguascaliente"

# Year range
START_YEAR = 2000
END_YEAR = 2022

# Paths
sites_csv = "/capstone/aridgw/data/2025_09_21_annual_median_groundwater_levels_16sites_tile_id.csv"
modis_data_dir = "/capstone/aridgw/data/modis_data/"
output_csv = "/capstone/aridgw/data/et_timeseries_aguascaliente_test.csv"
merged_output_csv = "/capstone/aridgw/data/et_gw_merged_aguascaliente_test.csv"  # Output for merged ET and GW data
# Buffer radius in meters
BUFFER_RADIUS = 1000  # 1 km

# MODIS constants
TILE_SIZE = 1111950.0
X_MIN = -20015109.354
Y_MAX = 10007554.677

In [3]:
# Real MODIS filename example 
# MOD16A3GF.A2000001.h00v08.061.2020264071747.hdf
# MOD16A3GF MODIS data naming convention 
# 2000 Year 
# Day of year 001
# h00v08 Tile h=0, v=8
# 061 = Collection number
# 2020264071747 = Production date
# {} Ensures there is a match for the specified number of digits
# ({}) Captures name as a group after ensuring there is a match


In [4]:
# Functions used in processing

def parse_modis_filename(filename):
    """Extract year, tile, h, v from MODIS filename"""
    # Naming convention first 
    pattern = r'MOD16A3GF\.A(\d{4})\d{3}\.h(\d{2})v(\d{2})\.\d{3}\.\d+\.hdf' # Defines regex pattern for MODIS filenames containing time and tile info
    match = re.search(pattern, filename) # Looks through modis_data_dir for files that match the pattern
    
    if match:
        return { # Returns a dictionary with extracted time and tile info
            'year': int(match.group(1)),
            'tile': f"h{match.group(2)}v{match.group(3)}",
            'h': int(match.group(2)),
            'v': int(match.group(3))
        }
    return None

def create_circle_buffer(center_lat, center_lon, radius_meters, target_crs):
    """Create circular buffer around site point
        - Input lat/lon in degrees
        - radius in meters
        - target_crs: pyproj CRS object for target projection (MODIS Sinusoidal)
    """
    point = gpd.GeoSeries([Point(center_lon, center_lat)], crs="EPSG:4326") # CRS is EPSG:4326 by default since lat/lon are used
    point_projected = point.to_crs(epsg=3857) # Project to Web Mercator for accurate buffering in meters
    circle = point_projected.buffer(radius_meters) # Create buffer in meters
    circle_target = circle.to_crs(target_crs) # Convert to target CRS (MODIS Sinusoidal)
    return circle_target

def extract_et_from_buffer(hdf_file, circle_geom, h, v):
    """Extract mean ET from buffer
    - Inputs:
        - hdf_file: path to MODIS HDF file
        - circle_geom: output geometry of create_circle_buffer function in MODIS CRS
        - h, v: tile indices
    - Returns:
        - mean ET value within buffer (float, mm/yr), or np.nan if no data
    """
    try:
        # Open HDF
        hdf = SD(hdf_file, SDC.READ)
        et_dataset = hdf.select('ET_500m')
        scale_factor = et_dataset.scale_factor
        fill_value = et_dataset.attributes()['_FillValue'] # Fill values should have been identified, they will turn into NaN
        
        # Read and clean data
        et_data_raw = et_dataset[:].astype(float) # Turn into float for NaN handling
        et_data_raw[et_data_raw == fill_value] = np.nan # Set fill values to NaN
        et_data_raw[et_data_raw > 65500] = np.nan # Additional cleaning for unrealistic values
        et_data = et_data_raw * scale_factor # Scale to (mm/yr) units using scaling factor
        
        # Setup geotransform
        PIXELS = et_data.shape[0] # Number of pixels in one dimension (assuming square)
        RES = TILE_SIZE / PIXELS # Calculate resolution, how many meters per pixel
        x_ul = X_MIN + h * TILE_SIZE # Upper-left x coordinate
        y_ul = Y_MAX - v * TILE_SIZE # Upper-left y coordinate
        transform = from_origin(x_ul, y_ul, RES, RES) # Create transformation
        modis_crs = CRS.from_proj4("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext") # MODIS Sinusoidal CRS
        
        # Create profile and clip
        profile = {
            "driver": "GTiff",
            "height": et_data.shape[0], # Pixels in y direction
            "width": et_data.shape[1], # Pixels in x direction
            "count": 1, # of bands
            "dtype": "float32", # Data type
            "crs": modis_crs, # CRS info
            "transform": transform, # Transformation info
            "nodata": np.nan
        }
        
        with MemoryFile() as memfile: # Use in-memory file for rasterio operations for efficiency
            with memfile.open(**profile) as dataset:
                dataset.write(et_data.astype("float32"), 1)
                clipped, _ = mask(dataset, circle_geom.geometry, crop=True) # Clip to buffer area
        
        # Calculate mean
        clipped_valid = clipped[0][~np.isnan(clipped[0])] # Valid (non-NaN) values only
        # Conduct mean zonal statistics
        mean_et = clipped_valid.mean() if len(clipped_valid) > 0 else np.nan # Mean ET or NaN if no valid data
        
        hdf.end()
        return mean_et
        
    except Exception as e:
        print(f"  Error: {e}")
        return np.nan


In [5]:
# Use CSV file with corresponding tile IDs for each site to extract ET timeseries

def extract_et_timeseries_single_site():
    """Extract annual ET timeseries for a single site using existing CSV"""
    
    print("="*70)
    print(f"Extracting ET timeseries for: {SITE_NAME}")
    print(f"Years: {START_YEAR}-{END_YEAR}")
    print("="*70)
    
    # STEP 1: Load site info from existing CSV

    print("\n STEP 1: Loading site information from CSV")
    
    df_sites = pd.read_csv(sites_csv)
    
    # Filter to just this site
    site_data = df_sites[df_sites['Title Location'] == SITE_NAME].iloc[0]
    
    site_lat = site_data['Lat']
    site_lon = site_data['Lon']
    tile_id = site_data['tile_id']
    
    print(f"✓ Location: {SITE_NAME}")
    print(f"✓ Coordinates: ({site_lat:.6f}, {site_lon:.6f})")
    print(f"✓ MODIS Tile: {tile_id}")
    
    # Extract h and v from tile_id (e.g., "h08v05" -> h=8, v=5)
    h = int(tile_id[1:3])
    v = int(tile_id[4:6])
    
    # STEP 2: Create buffer (One at time per site)

    print(f"\n STEP 2: Creating {BUFFER_RADIUS}m buffer")
    
    modis_crs = CRS.from_proj4("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")
    circle_geom = create_circle_buffer(site_lat, site_lon, BUFFER_RADIUS, modis_crs)
    print(f"✓ Buffer created")
    
    # STEP 3: Find and filter MODIS files according to tile and year range

    print(f"\n STEP 3: Finding MODIS files for tile {tile_id}")
    
    hdf_files = glob.glob(os.path.join(modis_data_dir, "*.hdf")) # Creates variable containing all .hdf files in modis_data_dir
    print(f"✓ Found {len(hdf_files)} total MODIS files")
    
    # Parse and filter files for this tile and year range
    file_dict = {}  # year -> filepath
    for hdf_file in hdf_files: # Looks through all .hdf files in modis_data_dir
        info = parse_modis_filename(os.path.basename(hdf_file)) # Parses filename to extract time and tile info
        if info and info['tile'] == tile_id and START_YEAR <= info['year'] <= END_YEAR: # Filters files for specified tile and year range
            file_dict[info['year']] = {'filepath': hdf_file, 'h': info['h'], 'v': info['v']} # Stores filtered files in dictionary with year as key
    
    print(f"✓ Filtered to {len(file_dict)} files for {tile_id} ({START_YEAR}-{END_YEAR})")
    
    if len(file_dict) == 0: # If no files found for specified tile and year range
        print(f"\n WARNING: No files found for tile {tile_id} in year range")
        print("Available tiles in directory:")
        for hdf_file in hdf_files[:5]:  # Show first 5 as examples
            info = parse_modis_filename(os.path.basename(hdf_file))
            if info: 
                print(f"  - {info['tile']} (year {info['year']})")
        return None, None
    
    # STEP 4: Extract ET for each year

    print(f"\n STEP 4: Extracting ET values") 
    print(f"\n{'Year':<8} {'Mean ET (mm/yr)':<20} {'Status'}")
    print("-" * 50) 
    
    results = []
    for year in range(START_YEAR, END_YEAR + 1): # Loops through each year in specified range
        if year in file_dict: 
            file_info = file_dict[year]
            mean_et = extract_et_from_buffer(
                file_info['filepath'],
                circle_geom,
                file_info['h'],
                file_info['v']
            )
            
            status = "✓" if not np.isnan(mean_et) else "NO DATA"
            et_display = f"{mean_et:.2f}" if not np.isnan(mean_et) else "NaN"
            print(f"{year:<8} {et_display:<20} {status}")
            
            results.append({ # Appends results to list
                'year': year,
                'location': SITE_NAME,
                'latitude': site_lat,
                'longitude': site_lon,
                'tile': tile_id,
                'mean_ET_mm_yr': mean_et,
                'buffer_radius_m': BUFFER_RADIUS
            })
        else:
            print(f"{year:<8} {'---':<20} FILE MISSING")
            results.append({
                'year': year,
                'location': SITE_NAME,
                'latitude': site_lat,
                'longitude': site_lon,
                'tile': tile_id,
                'mean_ET_mm_yr': np.nan,
                'buffer_radius_m': BUFFER_RADIUS
            })
    
    # STEP 5: Save ET results

    print("\n" + "="*70)
    print("STEP 5: Saving ET results")
    print("="*70)
    
    df_et = pd.DataFrame(results)
    df_et = df_et.sort_values('year')
    df_et.to_csv(output_csv, index=False)
    
    print(f"✓ ET data saved to: {output_csv}")
    print(f"\n ET Statistics:")
    print(f"  Years with data: {df_et['mean_ET_mm_yr'].notna().sum()} / {len(df_et)}")
    
    if df_et['mean_ET_mm_yr'].notna().sum() > 0:
        print(f"  Mean ET: {df_et['mean_ET_mm_yr'].mean():.2f} mm/yr")
        print(f"  Min ET: {df_et['mean_ET_mm_yr'].min():.2f} mm/yr")
        print(f"  Max ET: {df_et['mean_ET_mm_yr'].max():.2f} mm/yr")
        print(f"  Std Dev: {df_et['mean_ET_mm_yr'].std():.2f} mm/yr")
    
    # STEP 6: JOIN with groundwater data

    print("\n" + "="*70)
    print("STEP 6: Joining ET data with groundwater data")
    print("="*70)
    
    # Load the full groundwater dataset
    df_gw = pd.read_csv(sites_csv)
    
    # Filter to only this site
    df_gw_site = df_gw[df_gw['Title Location'] == SITE_NAME].copy()
    
    print(f"✓ Loaded {len(df_gw_site)} groundwater records for {SITE_NAME}")
    
    # Perform the join
    # Join on: location (from ET) = Title Location (from GW) AND year (ET) = YEAR (GW)
    df_merged_all_years = pd.merge(
        df_gw_site,
        df_et,
        left_on=['Title Location', 'YEAR'],
        right_on=['location', 'year'],
        how='left'  # Keep all groundwater records, add ET where available
    )
    # Filter to specified year range
    print(f"\n Filtering to years {START_YEAR}-{END_YEAR}...")

    df_merged = df_merged_all_years[ # Filters merged dataframe to specified year range
    (df_merged_all_years['YEAR'] >= START_YEAR) & 
    (df_merged_all_years['YEAR'] <= END_YEAR)
    ].copy()

    print(f"✓ Merged dataset has {len(df_merged)} rows")
    print(f"  Rows with ET data: {df_merged['mean_ET_mm_yr'].notna().sum()}")
    print(f"  Rows without ET data: {df_merged['mean_ET_mm_yr'].isna().sum()}")
    
    # Save merged dataset
    df_merged.to_csv(merged_output_csv, index=False)
    print(f"✓ Merged data saved to: {merged_output_csv}")
    
    # Show column comparison for verification
    print("\n📋 Column verification (duplicate columns help verify alignment):")
    verification_cols = ['Title Location', 'location', 'YEAR', 'year', 
                        'Lat', 'latitude', 'Lon', 'longitude', 
                        'tile_id', 'tile']
    available_cols = [col for col in verification_cols if col in df_merged.columns]
    if available_cols:
        print(df_merged[available_cols].head(5).to_string(index=False))
    
    return df_et, df_merged

# RUN
if __name__ == "__main__":
    df_et, df_merged = extract_et_timeseries_single_site()
    
    if df_et is not None and df_merged is not None:
        print("\n" + "="*70)
        print("ET OUTPUT PREVIEW")
        print("="*70)
        print(df_et.head(10).to_string(index=False))
        
        print("\n" + "="*70)
        print("MERGED OUTPUT PREVIEW")
        print("="*70)
        print(df_merged.head(10).to_string(index=False))
        
        print("\n" + "="*70)
        print("PROCESSING COMPLETE!")
        print("="*70)
        print(f"ET timeseries: {output_csv}")
        print(f"Merged data: {merged_output_csv}")


Extracting ET timeseries for: MexicoAguascaliente
Years: 2000-2022

 STEP 1: Loading site information from CSV
✓ Location: MexicoAguascaliente
✓ Coordinates: (22.198160, -102.310000)
✓ MODIS Tile: h08v06

 STEP 2: Creating 1000m buffer
✓ Buffer created

 STEP 3: Finding MODIS files for tile h08v06
✓ Found 7139 total MODIS files
✓ Filtered to 23 files for h08v06 (2000-2022)

 STEP 4: Extracting ET values

Year     Mean ET (mm/yr)      Status
--------------------------------------------------
2000     240.62               ✓
2001     269.75               ✓
2002     351.98               ✓
2003     370.08               ✓
2004     392.17               ✓
2005     318.19               ✓
2006     353.13               ✓
2007     380.20               ✓
2008     328.05               ✓
2009     309.63               ✓
2010     348.26               ✓
2011     230.88               ✓
2012     250.20               ✓
2013     342.01               ✓
2014     341.33               ✓
2015     407.38         